# Single iteration:

In [34]:
import pandas as pd
import altair as alt
import numpy as np

In [314]:
df = pd.read_csv('../processed_data/D4_single_0.4.csv')
#df = pd.read_csv('../processed_data/AmpC_single_0.4.csv')

print(len(df))
df['gain'] = df['N hits wanted']/0.004 / df['N hits explored']
df['Days'] = df['N hits explored'] / 60 / 60 /24
df.head()

147


,Unnamed: 0,Algorithm,Training size,N hits wanted,N hits explored,gain,Days
0,0,morgan_feat,5000,10000,289976,8.621403,3.356204
1,1,morgan_feat,10000,10000,221687,11.277161,2.565822
2,2,morgan_feat,20000,10000,154451,16.186363,1.787627
3,3,morgan_feat,40000,10000,150189,16.645693,1.738299
4,4,morgan_feat,80000,10000,146393,17.077319,1.694363


In [402]:
np.var([1,2,3])/np.sqrt(2)

0.4714045207910316

In [401]:
tstd([1,2,3])

1.0

In [381]:
d = pd.DataFrame({'a':[0.5,0.5,0.5,1.5,1.5,1.5], 'b':[1,2,3, 4,6,8]})

eb = alt.Chart(d).mark_errorbar(extent='ci').encode(
    x=alt.X('a:Q'),
    y=alt.Y('b:Q'),
    color=alt.Color('a:N')
)
pts = alt.Chart(d).mark_point().encode(
    x=alt.X('a:Q'),
    y=alt.Y('b:Q', aggregate='mean'),
    color=alt.Color('a:N')
)
eb+pts


alt.LayerChart(...)

In [387]:
line = alt.Chart(df).mark_line(color='black',size=2,opacity=0.5).encode(
  x=alt.X('Training size:Q', scale=alt.Scale(type='log')),
  y=alt.Y('gain',aggregate='mean',title='Enrichment'),
    color=alt.Color('N hits wanted:N')
)
pts = alt.Chart(df).mark_point().encode(
  x=alt.X('Training size:Q'),
  y=alt.Y('gain',aggregate='mean',),
    color=alt.Color('N hits wanted:N')
)

error_bars = alt.Chart(df).mark_errorbar(extent='stdev').encode(
  x=alt.X('Training size:Q',),
  y=alt.Y('gain', aggregate='mean'),
    color=alt.Color('N hits wanted:N')
)
#pts
line+pts+error_bars


alt.LayerChart(...)

In [319]:
line = alt.Chart(df).mark_line(color='black',size=2,opacity=0.5).encode(
  x=alt.X('Training size:Q', scale=alt.Scale(type='log')),
  y=alt.Y('Days',aggregate='mean',title='Days'),
    color=alt.Color('N hits wanted:N')
)
pts = alt.Chart(df).mark_point(filled=False,size=65).encode(
  x=alt.X('Training size:Q'),
  y=alt.Y('Days',aggregate='mean',),
    color=alt.Color('N hits wanted:N')
)

error_bars = alt.Chart(df).mark_errorbar(extent='ci').encode(
  x=alt.X('Training size:Q',),
  y=alt.Y('Days'),
    color=alt.Color('N hits wanted:N')
)
#pts
line+pts+error_bars


alt.LayerChart(...)

# 'Active learning" approach plot

In [140]:
!ls ../processed_data/D4_recon*

../processed_data/D4_reconstruction_0.125.csv
../processed_data/D4_reconstruction_0.15.csv
../processed_data/D4_reconstruction_0.175.csv
../processed_data/D4_reconstruction_1.5.csv
../processed_data/D4_reconstruction_1.75.csv


In [204]:
df = pd.read_csv('../processed_data/ampc_reconstruction_0.125.csv')
df['Algorithm'] = 'AmpC, LogReg (ours)'
df_ = pd.read_csv('../processed_data/D4_reconstruction_0.15.csv')
df_['Algorithm'] = 'D4: LogReg (ours)'

In [205]:
df = pd.concat([df_, df])

In [206]:
prev_results = [['AmpC:RF (Graff)', 400_000, 71.4, 2.1], ['AmpC:NN (Graff)', 400_000, 74.7, 1.4],
                ['AmpC:MPN (Graff)',400_000, 87.9, 2.3],
    ['AmpC:RF (Graff)', 200_000, 45.5, 1.8],
['AmpC:NN (Graff)', 200_000, 52.8, 0.5],
['AmpC:MPN (Graff)', 200_000, 67.1, 2.1],
['AmpC:RF (Graff)', 100_000, 24.0, 2.2],
['AmpC:NN (Graff)', 100_000 , 33.3,0.3],
['AmpC:MPN (Graff)', 100_000, 52.0, 0.5]]


In [207]:


coley = pd.DataFrame(columns=['Algorithm', 'Training size', 'N ligands explored', '% top-k found'])
count = 0 
for res in prev_results:
    
    desired_std_dev = res[3]
    samples = np.array([-1,0,1]).astype(float)
    samples *= (desired_std_dev/np.std(samples))
    for s in samples:
        coley.loc[count]= [res[0], res[1], res[1]*6, (s+res[2])/100]
        count+=1

In [208]:
concat = pd.concat([df, coley])

In [209]:
error_bars = alt.Chart(concat).mark_errorbar(extent='ci').encode(
  x=alt.X('N ligands explored:Q',title='Number of ligands sampled'),
  y=alt.Y('% top-k found:Q', title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

points = alt.Chart(concat).mark_point(filled=True, color='black').encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

line = alt.Chart(concat).mark_line(color='black',size=1,opacity=0.5).encode(
  x=alt.X('N ligands explored:Q'),
  y=alt.Y('% top-k found:Q',aggregate='mean',title='% top 50,000 found'),
    color=alt.Color('Algorithm')
)

ch = (error_bars+points+line).properties(height=400,width=200).facet(
    column=alt.Column('Training size:N',sort=alt.Sort([0.004, 0.002, 0.001])),
).resolve_scale(x='independent')
ch

alt.FacetChart(...)

In [102]:
#df.to_csv('../processed_data/ampc_reconstruction.csv')

In [103]:
chs = []
for frac in [400000, 200000, 100000]:
    
    df_ = concat[concat['Training size']==frac].replace('morgan_feat', 'Morgan pharm. & Log.reg. (ours)')
    error_bars = alt.Chart(df_).mark_errorbar(extent='ci').encode(
          x=alt.X('N ligands explored:Q', 
                  title='Number of ligands sampled',
                  scale=alt.Scale(domain=[0,max(df_['N ligands explored'])+10000])),
          y=alt.Y('% top-k found:Q',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    points = alt.Chart(df_).mark_point(filled=True, color='black').encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )

    line = alt.Chart(df_).mark_line(color='black',size=1,opacity=0.5).encode(
          x=alt.X('N ligands explored:Q'),
          y=alt.Y('% top-k found:Q',aggregate='mean',scale=alt.Scale(domain=[0,0.95])),
        color=alt.Color('Algorithm')
        )
    ch = (error_bars+points+line).properties(width=200)
    ch.title = str(frac / (100*1e6)*100)
    chs.append(ch)


In [104]:
chs[0]
sup = chs[0] |  chs[1] | chs[2]
sup
#sup.save('../figures/ampC_reconstruction.html') #using 0.05

alt.HConcatChart(...)

In [109]:
chs[0]+chs[1]+chs[2]

alt.LayerChart(...)

In [289]:
#sup.save('../figures/ampC_reconstruction.html')

In [28]:
!pwd

/home/lewis/Documents/projects/dockop/code
